In [1]:
with open("input.txt", "r", encoding="utf-8") as f:
    text = f.read()

In [2]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [3]:
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("hello world"))
print(decode(encode("hello world")))

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]
hello world


In [4]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])


torch.Size([1115393]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [5]:
# split training and validation data
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [6]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [7]:
# explanation (not needed)
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target is:{target}")

when input is tensor([18]) the target is:47
when input is tensor([18, 47]) the target is:56
when input is tensor([18, 47, 56]) the target is:57
when input is tensor([18, 47, 56, 57]) the target is:58
when input is tensor([18, 47, 56, 57, 58]) the target is:1
when input is tensor([18, 47, 56, 57, 58,  1]) the target is:15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is:47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is:58


In [8]:
torch.manual_seed(1337)
batch_size = 4 # how many independant sequences will be processed in parallel
block_size = 8 # what is the maximum context length for predictions

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('---')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"when input is {context.tolist()} the target is: {target}")


inputs
torch.Size([4, 8])
tensor([[53, 59,  6,  1, 58, 56, 47, 40],
        [49, 43, 43, 54,  1, 47, 58,  1],
        [13, 52, 45, 43, 50, 53,  8,  0],
        [ 1, 39,  1, 46, 53, 59, 57, 43]])
targets:
torch.Size([4, 8])
tensor([[59,  6,  1, 58, 56, 47, 40, 59],
        [43, 43, 54,  1, 47, 58,  1, 58],
        [52, 45, 43, 50, 53,  8,  0, 26],
        [39,  1, 46, 53, 59, 57, 43,  0]])
---
when input is [53] the target is: 59
when input is [53, 59] the target is: 6
when input is [53, 59, 6] the target is: 1
when input is [53, 59, 6, 1] the target is: 58
when input is [53, 59, 6, 1, 58] the target is: 56
when input is [53, 59, 6, 1, 58, 56] the target is: 47
when input is [53, 59, 6, 1, 58, 56, 47] the target is: 40
when input is [53, 59, 6, 1, 58, 56, 47, 40] the target is: 59
when input is [49] the target is: 43
when input is [49, 43] the target is: 43
when input is [49, 43, 43] the target is: 54
when input is [49, 43, 43, 54] the target is: 1
when input is [49, 43, 43, 54, 1] the 

In [20]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token reads off the logits for the next token in the lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both (B, T) tensor if ints
        logits = self.token_embedding_table(idx) # (B, T, C)

        if targets == None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx os (B, T) array of indices in current context
        for _ in range(max_new_tokens):
            # get predictions
            logits, loss = self(idx)
            # focus only on last timestep
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=1) # (B, C)
            # sample from distribution
            idx_next = torch.multinomial(probs, num_samples=1)
            # append sampled index to running sequence
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

    
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)
# calculate ideal loss -ln(1/65)

# idx = torch.zeros((1, 1), dtype=torch.long)  # 1x1 tensor holding a 0 - kicks off generation
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))
    


torch.Size([32, 65])
tensor(4.8948, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [46]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [59]:
# run to train
batch_size = 32
for steps in range(1000):
    # sample data
    xb, yb = get_batch('train')

    # calculate loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


2.394850492477417


In [60]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


S:
We mbouk'su mbam

ERCOMIUCoake:
them man.
BEYOMETht l at
Bun st tite wit we t mpt mamas lissherrd


In [61]:
# mathematical trick in self-attention

torch.manual_seed(1337)
B, T, C = 4, 8, 2 # batch time channels
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [ ]:
# version 1
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1]
        xbow[b, t] = torch.mean(xprev, 0)

print(x[0])
print(xbow[0])

In [70]:
# version 2
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x
torch.allclose(xbow, xbow2)

True

In [71]:
# using softmax 
# overall this adds the weights for the past elements
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf')) # make all 0s -infinity
wei = F.softmax(wei, dim=-1)
xbow2 = wei @ x
torch.allclose(xbow, xbow2)

True

In [76]:
torch.manual_seed(1337)
B, T, C = 4, 8, 2 # batch time channels
x = torch.randn(B, T, C)

# single headf performing self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False) # if key and query align then those characters are related more
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) # (B, T, 16)
q = query(x) # (B, T, 16)
wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) --> (B, T, T) OR dot product of query and key 

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T, T)) # initialise uniform weights between tokens
wei = wei.masked_fill(tril == 0, float('-inf')) # make all 0s -infinity (if line deleted then all nodes could talk to all other nodes - present in decoder blocks absent in encoder blocks)
wei = F.softmax(wei, dim=-1) 

v = value(x)
out = wei @ v

out.shape
wei

tensor([[[1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00],
         [5.5992e-01, 4.4008e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00],
         [3.2197e-01, 2.0162e-01, 4.7641e-01, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00],
         [1.6397e-01, 8.1458e-02, 2.9607e-01, 4.5850e-01, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00],
         [2.0508e-01, 3.0070e-01, 1.8936e-01, 1.8076e-01, 1.2409e-01,
          0.0000e+00, 0.0000e+00, 0.0000e+00],
         [6.0019e-02, 1.2732e-01, 2.9082e-02, 1.6934e-02, 5.5211e-02,
          7.1144e-01, 0.0000e+00, 0.0000e+00],
         [1.4085e-01, 1.0251e-01, 1.7444e-01, 2.0379e-01, 1.6895e-01,
          6.6888e-02, 1.4256e-01, 0.0000e+00],
         [2.2274e-02, 1.0857e-01, 8.2257e-03, 4.0054e-03, 8.0379e-03,
          7.2572e-01, 2.1608e-02, 1.0156e-01]],

        [[1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.00

In [77]:
class BatchNorm1d:
    def __init__(self, dim, eps=1e-5, momentum=0.1):
        self.eps = eps
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)

    def __call__(self, x):
        xmean = x.mean(1, keepdim=True) # batch mean - columns=1, rows=0
        xvar = x.var(1, keepdim=True) # batch varience
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps)
        self.out = self.gamma * xhat + self.beta
        return self.out
    
    def parameters(self):
        return [self.gamma, self.beta]
    
torch.manual_seed(1337)
module = BatchNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimension
x = module(x)
x.shape

torch.Size([32, 100])

In [67]:
# demo
torch.tril(torch.ones(3, 3))
# generates triangular bottom half 1s and rest 0s

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [69]:
# demo
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3)) # use tril to sum only current int and past ints
a = a / torch.sum(a, 1, keepdim=True) # normalise a tensor so each row sums to 1 - allows you to get average incrementally
b = torch.randint(0, 10,(3,2)).float()
c = a @ b # dot product / matrix multiplication
print('a=')
print(a)
print('---')
print('b=')
print(b)
print('---')
print('c=')
print(c)
print('---')

# when multiplied by a (all 1s) it just does a sum of the columns

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
---
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
---
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])
---
